In [1]:
import asyncio
import difflib
import json
import sys
from functools import wraps

import pandas as pd
import plotly.io
from litellm import acompletion, completion_cost
from pydantic import BaseModel
from tqdm.asyncio import tqdm
from unidecode import unidecode

sys.path.append("../..")

plotly.io.renderers.default = "png"

## Make predictions for one month (07/2024)


In [3]:
# Load data
df = pd.read_parquet("../../data/raw/18_channels_2023_09_to_2024_09.parquet")

# Filter some channels only
df = df[df["channel_name"].isin(["sud-radio", "europe1", "itele"])]
# df = df[(df["start"] >= "2024-07-01") & (df["start"] < "2024-08-01")]

# Fixing some syntax issues in the data
# Mediatree issue with "' "
df["text"] = df["text"].str.replace("' ", "'")

df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
1dcd4b454f8bac42440259ce26a1a2192051186bb5728be489dc654a9a967d1d,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383
0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc93f701fd56bd8ea66f,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593
b6d54aefb250671e7754a688411ce9e68badcad88a665be3de78996d13b74fd2,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606
23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d6659accf565ba61f48ae,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515
71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce412813a06a60a2e47d3c,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536
...,...,...,...,...,...,...,...,...,...,...
121c2c2780f373e0d03ad266a5b4c11e87873e353954051693a978db123e1a04,2024-09-12 07:06:00,est la période la plus à risque pour les inond...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""adaptation_climatique_solutions"", ""changemen...","[{""keyword"": ""inondation"", ""timestamp"": 172611...",2,575
10e4d16cad946340c67738bdaa487603d4635c6d2db978b63aff13916a49cfab,2024-09-12 08:00:00,solaire point fr votre expert en panneaux sola...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,526
74f1b824b772877e6e05908a308a8ae31146b652ff50793bbd7fc9d4deb5e29c,2024-09-12 09:00:00,foyer trois mille neuf cent quatre-vingt-dix e...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,520


In [15]:
from openai import AsyncOpenAI

client = AsyncOpenAI()

# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(100)


# Decorator that ensures `acompletion` uses the semaphore
def with_semaphore(acquire_semaphore):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            async with acquire_semaphore:
                return await func(*args, **kwargs)

        return wrapper

    return decorator


acompletion = with_semaphore(semaphore)(acompletion)


class MediatreePrediction(BaseModel):
    label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


class Claim(BaseModel):
    claim: str
    context: str
    analysis: str
    disinformation_score: str
    disinformation_category: str
    pro_anti: str
    speaker: str
    contradiction: str
    quote: str


class Claims(BaseModel):
    claims: list[Claim]


async def detect_claims(transcription):
    prompt = f"""Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. 
A partir de cet extrait liste moi tous les faits/opinions environnementaux (claim) uniques qu'il faudrait factchecker. Et pour chaque claim, donne une première analyse si c'est de la désinformation ou non, un score si c'est de la désinformation, ainsi qu'une catégorisation de cette allégation.
Ne sélectionne que les claims sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ..) et pas sur les thématiques sociales et/ou économiques
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois les champs suivants : 

- "claim" - l'allégation à potentiellement vérifier
- "context" - reformulation du contexte dans laquelle cette allégation a été prononcée (maximum 1 paragraphe)
- "analysis" - première analyse du point de vue de l'expert sur le potentiel de désinformation de cette allégation en fonction du contexte
- "disinformation_score" - le score de désinformation (voir plus bas)
- "disinformation_category" - la catégorie de désinformation (voir plus bas)
- "pro_anti" - si l'allégation est plutôt anti-écologie ou pro-écologie
- "speaker" - nom et fonction de la personne qui a prononcé l'allégation si on a l'information (sinon "N/A")
- "contradiction" - si l'allégation a été contestée dans un dialogue, résume la contradiction (sinon "N/A")
- "quote" - la citation exacte qui correspond à l'allégation

Pour les scores "disinformation_score"
- "very low" = pas de problème, l'allégation n'est pas trompeuse ou à risque. pas besoin d'investiguer plus loin
- "low" = allégation qui nécessiterait une vérification et une interrogation, mais sur un sujet peu important et significatif dans le contexte des enjeux écologiques (exemple : les tondeuses à gazon, 
- "medium" = allégation problématique sur un sujet écologique important (scientifique, impacts, élections, politique, transport, agriculture, énergie, alimentation, démocratie ...) , qui nécessiterait vraiment d'être vérifiée, déconstruite, débunkée et interrogée. En particulier pour les opinions fallacieuses
- "high" = allégation grave, en particulier si elle nie le consensus scientifique

Pour les catégories de désinformation "disinformation_category": 
- "consensus" = négation du consensus scientifique
- "facts" = fait à vérifier, à préciser ou contextualiser
- "narrative" = narratif fallacieux ou opinion qui sème le doute (par exemple : "les écolos veulent nous enlever nos libertés")
- "other"

<transcription>
{transcription}
</transcription>
    """
    async with semaphore:
        # Retry if errors
        while True:
            try:
                completion = await client.beta.chat.completions.parse(
                    model="gpt-4o-mini",
                    messages=[
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ],
                    response_format=Claims,
                )
                break
            except Exception:
                print("Error, retrying...")

    n_tokens = completion.usage.total_tokens
    claims_data = completion.choices[0].message.parsed
    result = [claim.dict() for claim in claims_data.claims]

    return result

In [5]:
# Make predictions
mediatree_predictions: list[MediatreePrediction] = await tqdm.gather(
    *[detect_claims(text) for text in df["text"]]
)
# mediatree_predictions_df = pd.DataFrame(
#     [pred.model_dump(exclude_none=True) for pred in mediatree_predictions],
#     index=df.index,
# )
# df = pd.concat([df, mediatree_predictions_df], axis=1)

100%|██████████| 34519/34519 [18:25<00:00, 31.24it/s]  


In [29]:
df["claims"] = mediatree_predictions
# df["claims"] = df["claims"].apply(lambda x: x.to_dict(orient="records"))

In [7]:
df["label_pred"].value_counts()

label_pred
0_accepted                 12332
3_its_not_bad                513
1_its_not_happening          333
5_science_is_unreliable      266
4_solutions_wont_work        259
2_its_not_us                  57
Name: count, dtype: int64

In [8]:
(df["label_pred"] != "0_accepted").sum()

np.int64(1428)

In [33]:
# Save the result
df.to_parquet(
    "../../data/experiments/label_studio_review/3_channels_predictions_09_2023_09_2024.parquet"
)

In [37]:
(df["claims"].str.len() > 1).sum()

np.int64(14526)

In [54]:
claims_list = df["claims"].tolist()

In [55]:
for id, claims_sublist in zip(df.index, claims_list):
    for claim in claims_sublist:
        claim["id"] = id

In [56]:
c = [l for sl in claims_list for l in sl]

In [57]:
cdf = pd.DataFrame(c)

In [58]:
cdf

,claim,context,analysis,disinformation_score,disinformation_category,pro_anti,speaker,contradiction,quote,id
0,les initiatives positives de jean z une nova s...,L'interview mentionne des initiatives sur la g...,L'allégation semble promouvoir des pratiques p...,low,facts,pro-écologie,"Emmanuel Kessler, directeur de la rédaction du...",N/A,les initiatives positives de jean z une nova s...,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...
1,Les propriétaires de campings peuvent réaliser...,Le locuteur explique qu'une innovation dans le...,Cette affirmation est généralement vérifiable ...,low,facts,pro-écologie,N/A,N/A,la facture s'allège de manière non négligeable...,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...
2,Le nombre de campings équipés de douches conne...,Lors de la discussion sur les innovations dans...,Ceci est une information qui peut être vérifié...,low,facts,pro-écologie,N/A,N/A,le nombre de camping équipé a doublé depuis l'...,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...
3,Les limitations de douches ne sont plus vues c...,Le locuteur évoque un changement d'attitude de...,Cette observation peut refléter un phénomène s...,medium,narrative,pro-écologie,N/A,N/A,les propriétaires et les campeurs ne considère...,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...
4,Titeuf explique ce qu'est le bilan carbone.,Lors d'une discussion sur les sujets abordés d...,L'allégation sur le fait que Titeuf peut expli...,low,facts,pro-écologie,N/A,N/A,j'ai bien aimé dessinées titeuf qui explique c...,b6d54aefb250671e7754a688411ce9e68badcad88a665b...
...,...,...,...,...,...,...,...,...,...,...
59295,Il ne faut pas tenter d'aller chercher son enf...,"Eric Porte, à la tête du centre d'information ...",Cette recommandation s'inscrit dans une logiqu...,low,other,pro-écologie,"Eric Porte, centre d'information pour la préve...",N/A,Il ne faut pas tenter d'aller chercher son enf...,121c2c2780f373e0d03ad266a5b4c11e87873e35395405...
59296,Les baisses vitres qui sont électriques peuven...,"Christophe Mirmand, préfet de la région Paca, ...",Cette déclaration est réaliste et met en lumiè...,low,other,pro-écologie,"Christophe Mirmand, préfet de la région Paca",N/A,Les baisses vitres qui sont électriques peuven...,121c2c2780f373e0d03ad266a5b4c11e87873e35395405...
59297,les températures sont descendues entre trois e...,L'intervenant décrit les conditions climatique...,Cette allégation est factuelle et se réfère au...,very low,facts,N/A,"Rémi Fraisse, expert en panneaux solaires",N/A,les températures sont descendues entre trois e...,10e4d16cad946340c67738bdaa487603d4635c6d2db978...
59298,Les panneaux solaires permettent de faire des ...,Cette allégation a été faite dans le cadre d'u...,"Cette allégation est généralement exacte, car ...",very low,facts,pro-écologie,N/A,N/A,Les panneaux solaires permettent de faire des ...,74f1b824b772877e6e05908a308a8ae31146b652ff5079...


In [59]:
cdf = cdf[cdf["disinformation_score"] == "high"]

In [61]:
cdf["id"].nunique()

1560

In [12]:
def show_llm_usage(df: pd.DataFrame) -> None:
    print("\nLLM USAGE\n=========\n")
    print(
        f"Median token usage:\n- Prompt: {int(df["prompt_tokens"].median())}\n"
        f"- Completion: {int(df["completion_tokens"].median())}\n"
        f"- Total: {int(df["total_tokens"].median())}"
    )
    print(f"\nTotal cost: ${df["cost"].sum():.3f}\n")


show_llm_usage(df)


LLM USAGE

Median token usage:
- Prompt: 1402
- Completion: 27
- Total: 1433

Total cost: $3.126



In [13]:
df["label_pred"].value_counts()

label_pred
0_accepted                 12332
3_its_not_bad                513
1_its_not_happening          333
5_science_is_unreliable      266
4_solutions_wont_work        259
2_its_not_us                  57
Name: count, dtype: int64

## Manually review the predictions with LabelStudio


In [14]:
df = pd.read_csv(
    "../../data/experiments/label_studio_review/mediatree_predictions_07_2024.csv",
    index_col=0,
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,label_pred,prompt_tokens,completion_tokens,total_tokens,cost,claim_pred
id,,,,,,,,,,,,,,,,
ae3110362b2d90e3b39022926052cb8c57d52be76f692dfccb179236a64fdefd,2024-01-01 06:02:00,sur la boisson maintenant que les fêtes sont a...,europe1,True,Information - Magazine,Bonjour,"[""ressources"", ""changement_climatique_constat""...","[{""keyword"": ""transition \u00e9cologique"", ""ti...",1,524,0_accepted,1424,27,1451,0.000230,NaN
068edefaad4938b6cf7a86bf8e306f6968bbbf3c96e84627b6295900ed9d5615,2024-01-01 06:14:00,première prescription culture de l'année c'est...,europe1,True,Information - Magazine,Bonjour,"[""changement_climatique_constat"", ""biodiversit...","[{""keyword"": ""ademe"", ""timestamp"": 17040861260...",1,497,0_accepted,1392,27,1419,0.000225,NaN
f647d3fad5acbe70ee52b488bd3e8ec8be1018acc4e7756b48bb29b81b877e50,2024-01-01 06:36:00,hamas hier soir c'était donc un réveillon dans...,europe1,True,Information - Magazine,Bonjour,"[""attenuation_climatique_solutions"", ""biodiver...","[{""keyword"": ""sobri\u00e9t\u00e9"", ""timestamp""...",1,496,0_accepted,1397,27,1424,0.000226,NaN
d346e52092308430d6ec6faa668722219ce024db394c112e5e79ba9b8352ff89,2024-01-01 06:54:00,d'accélérer cette part du made in france mais ...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""attenuatio...","[{""keyword"": ""emballage"", ""timestamp"": 1704088...",1,488,0_accepted,1385,27,1412,0.000224,NaN
cb1c730c8eabec84989e98a517db234d22b495d3e0814a9a6286dd61f949d89c,2024-01-01 07:02:00,qui porte bonheur pour moi le quatre août deux...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""feux"", ""timestamp"": 170408896200...",1,528,0_accepted,1422,27,1449,0.000229,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32b48450801f677a3834cb84281b9d91d974f8c8d3ca7e84bed103d7218566de,2024-07-31 10:04:00,dans toute la france le bureau de recherches g...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""changement_climatique_constat"", ""changement_...","[{""keyword"": ""agriculteur"", ""timestamp"": 17224...",4,530,0_accepted,1424,27,1451,0.000230,NaN
2750001e70a1cd4eae8ac6932a495b7ac97feeab1e9d747a53a792b79d4d6a5d,2024-07-31 10:58:00,servi de point fr pourquoi venir chez castoram...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 172241628202...",1,523,0_accepted,1415,27,1442,0.000228,NaN
26b53013a4e78c907b56e884c01930963d66390e6d431463c87ea361cea08771,2024-07-31 11:00:00,triathlon et la dix neuvième médaille pour la ...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""changement_climatique_consequences"", ""biodiv...","[{""keyword"": ""alimentation"", ""timestamp"": 1722...",2,566,0_accepted,1460,27,1487,0.000235,NaN


In [15]:
# Generated code!
def approximate_match(text: str, claim: str, threshold=0.6) -> tuple[int, int] | None:
    # Preprocess text to remove casing and accents
    text = unidecode(text.lower())
    claim = claim.lower()

    # Split text and claim into words
    text_words = text.split()
    claim_words = claim.split()

    # Initialize variables to keep track of best match
    best_match_score = 0
    best_match_range = (None, None)

    # Slide a window of claim length over text to find the best match
    for i in range(len(text_words) - len(claim_words) + 1):
        # Extract a slice of words from the text to compare with claim
        window = text_words[i : i + len(claim_words)]

        # Calculate similarity ratio between the window and claim
        similarity = difflib.SequenceMatcher(None, window, claim_words).ratio()

        # Check if the similarity is above the threshold and better than the previous best
        if similarity > threshold and similarity > best_match_score:
            best_match_score = similarity
            best_match_range = (i, i + len(claim_words) - 1)

    # Convert word indices to character indices
    if best_match_range != (None, None):
        start_word_idx, end_word_idx = best_match_range
        start_char_idx = len(" ".join(text_words[:start_word_idx])) + (
            1 if start_word_idx > 0 else 0
        )
        end_char_idx = len(" ".join(text_words[: end_word_idx + 1]))
        return (start_char_idx, end_char_idx)
    else:
        return None

In [16]:
# Format for LabelStudio
# Only review flagged transcripts
data = []
for id, row in df[df["label_pred"] != "0_accepted"].iterrows():
    # Find the extracted claim in the original text
    text: str = row["text"]
    claim_pred: str = row["claim_pred"]
    try:
        # Try to see if we have a perfect match of claim in text
        start = text.lower().index(claim_pred.lower())
        end = start + len(claim_pred)
    except ValueError:
        # ValueError: substring not found
        matching_result = approximate_match(text, claim_pred, threshold=0.5)
        if matching_result:
            start, end = matching_result
        else:
            # The LLM "extracted" a claim that doesn't exist in the text...
            start, end = None, None

    # Pre-annotate the task with the model's prediction
    # We skip it if we couldn't match the extracted claim with the original text
    if start is not None:
        data.append(
            {
                "data": {
                    "id": id,
                    "start": str(row["start"]),
                    "text": text,
                    "channel_name": row["channel_name"],
                    "channel_program_type": row["channel_program_type"],
                    "channel_program": row["channel_program"],
                    "label_pred": row["label_pred"],
                    "claim_pred": claim_pred,
                },
                "predictions": [
                    {
                        "model_version": "claude-3-haiku",
                        "result": [
                            {
                                "value": {
                                    "start": start,
                                    "end": end,
                                    "text": text[start:end],
                                    "labels": [row["label_pred"]],
                                },
                                "from_name": "label",
                                "to_name": "text",
                                "type": "labels",
                            }
                        ],
                    }
                ],
            }
        )

In [17]:
# Save the LabelStudio tasks for import
with open(
    "../../data/experiments/label_studio_review/mediatree_predictions_07_2024_label_studio_review.json",
    "w",
) as f:
    json.dump(data, f)

In [13]:
len(data)

202